In [22]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [51]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,Jan-99,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [38]:
# These variables (columns) were eliminated form the dataset: 
# EIN and NAME columns are assumed to have no relationship to IS_SUCCESSFUL

# The following variables are defined as X (features): 
# APPLICATION_TYPE (17)
# AFFILIATION (6)
# CLASSIFICATION (71 - NEEDS BINNING - COMBINE C1245, C1283, C2570, C2561, C4500 and more into C_OTHER)
# USE_CASE (5)
# ORGANIZATION (4)
# SPECIAL_CONSIDERATIONS columns only had 5 and 27 rows respectively out of approximately 35,000.  
# INCOME_AMT (9) 
# STATUS (2)
# ASK_AMT (Integer) 

# Target variable = IS_SUCCESSFUL. This variable is already encoded as a 0 or 1. 

# Drop EIN and NAME columns

In [52]:
charity_df = charity_df.drop(columns=['EIN', 'NAME'])
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,Jan-99,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [53]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

In [54]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

# Encode the columns

In [55]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# MERGE Back Into DF

In [56]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,INCOME_AMT_Jan-99,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Set Y = Target and X = Features

In [44]:
# Remove loan status target from features data
y = charity_df.IS_SUCCESSFUL
X = charity_df.drop(columns=["IS_SUCCESSFUL"])



# Split Data (Train/Test) and Scale

In [45]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Select ML (RandomForest), Create Instance, Fit/Train, Predict

In [46]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")


 Random forest predictive accuracy: 0.699


# Select Deep Neural Network, Set Parameters, Fit/Train, Evaluate

In [12]:
# Using a TensorFlow neural network design of your choice, create a binary classification model
# that can predict if an Alphabet Soup funded organization will be successful based on the features in the dataset.
#    You may choose to use a neural network or deep learning model.
#    Hint: Think about how many inputs there are before determining the number of neurons and layers in your model.

# Compile, train, and evaluate your binary classification model. 
# Be sure that your notebook produces the following outputs:
#   Final model loss metric
#   Final model predictive accuracy

In [47]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()



Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 24)                1032      
_________________________________________________________________
dense_19 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 13        
Total params: 1,345
Trainable params: 1,345
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.6076 - accuracy: 0.7052
Epoch 2/50
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5717 - accuracy: 0.7272
Epoch 3/50
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5686 - accuracy: 0.7282
Epoch 4/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5674 - accuracy: 0.7285
Epoch 5/50
25724/25724 [==============================] - 1s 37us/sample - loss: 0.5667 - accuracy: 0.7280
Epoch 6/50
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5662 - accuracy: 0.7290
Epoch 7/50
25724/25724 [==============================] - 1s 40us/sample - loss: 0.5652 - accuracy: 0.7299
Epoch 8/50
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5654 - accuracy: 0.7279
Epoch 9/50
25724/25724 [==============================] - 1s 36us/sample - loss: 0.5652 - accuracy: 0.7295
Epoch 10/50
25

In [ ]:
#Optimization Results
# * Eliminated 2 more columns Status and Special_Considerations: Loss 0.5340 Acc: 0.7401
# Added CLASSIFICATION to columns eliminated: Loss 0.5611 Acc: 0.7308        

# Optimize the model with different configurations

In [14]:
# Do your best to optimize your model training and input data to achieve a
# target predictive accuracy higher than 75%.
#   Look at Page 19.2.6 for ideas on how to optimize and boost model performance.
#   Note: You will not be penalized if your model does not achieve target performance,
#   as long as you demonstrate an attempt at model optimization within your notebook.

In [20]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  48
hidden_nodes_layer2 =24

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()




Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 48)                5616      
_________________________________________________________________
dense_13 (Dense)             (None, 24)                1176      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 25        
Total params: 6,817
Trainable params: 6,817
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=300)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


In [16]:
# Create a new README.txt file within the same folder as your AlphabetSoupChallenge.ipynb notebook.
# Include a 5–10 sentence writeup in your README that addresses the following questions:
#   How many neurons and layers did you select for your neural network model? Why?
#   Were you able to achieve the target model performance? What steps did you take to try and
#   increase model performance?
#   If you were to implement a different model to solve this classification problem, which would you choose? Random Forest. Why?

In [17]:
# Make sure your repo is up to date and includes the following:

#     An AlphabetSoupChallenge.ipynb file containing all of your model building code
#     A README.txt file containing your writeup
#     Submit a link to your repository through Canvas.